In [10]:
from binance.client import Client
import pandas as pd

In [53]:
api_key = "dLgGHojVxEF3ulipNtYNSh4wKacDnZjkzOhR5sbx431ditIjgF7iqbbLniAFmUQ6"
secret_key = pd.read_table("secret_key.txt").columns[0]

'I3SQm1Wy4PueYdqIm8dBY5EQTZbN08awRieafAU24U6aRLKILGDX3FxnB66ESm6i'

In [54]:
client = Client(api_key = api_key, api_secret = secret_key, tld = "com")

In [55]:
client

In [56]:
client.ping()

{}

In [57]:
client.get_system_status()

{'status': 0, 'msg': 'normal'}

In [58]:
account = client.get_account()
account

{'makerCommission': 10,
 'takerCommission': 10,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00100000',
  'taker': '0.00100000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1685071800434,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BTC', 'free': '0.00046953', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.03997000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {

In [17]:
account["accountType"]

NameError: name 'account' is not defined

In [18]:
pd.to_datetime(account["updateTime"], unit="ms")

NameError: name 'account' is not defined

In [ ]:
account["balances"]

In [ ]:
df = pd.DataFrame(account["balances"])
df

In [ ]:
df.info()

In [ ]:
df.free = pd.to_numeric(df.free, errors="coerce")
df.locked = pd.to_numeric(df.locked, errors="coerce")

In [ ]:
df.loc[df.free > 0]

In [ ]:
client.get_asset_balance(asset="BTC")

In [ ]:
client.get_asset_balance(asset="ETH")

In [ ]:
float(client.get_asset_balance(asset="EUR")["free"])

In [ ]:
snap = client.get_account_snapshot(type="SPOT")
snap

In [ ]:
snap = pd.json_normalize(snap["snapshotVos"])
snap

In [ ]:
snap.updateTime = pd.to_datetime(snap["updateTime"], unit="ms")
snap

In [ ]:
client.get_exchange_info()["rateLimits"]

In [ ]:
client.get_all_coins_info()

In [ ]:
coins = pd.DataFrame(client.get_all_coins_info())
coins

In [ ]:
coins.loc[coins.coin == 'BTC']

In [ ]:
coins.loc[coins.coin == 'EUR']

## 현재 시장정보 얻기

In [ ]:
client.get_trade_fee(symbol = "BTCUSDT")

In [ ]:
client.get_trade_fee(symbol = "BTCEUR")

In [ ]:
client.get_trade_fee(symbol = "BUSDUSDT")

In [ ]:
client.get_symbol_info(symbol = "BTCEUR")

In [ ]:
client.get_symbol_ticker(symbol = "BTCUSDT")

In [ ]:
float(client.get_symbol_ticker(symbol = "BTCUSDT")["price"])

In [ ]:
client.get_avg_price(symbol = 'BTCUSDT')

In [ ]:
prices = client.get_all_tickers()
prices

In [ ]:
df = pd.DataFrame(prices)
df

In [ ]:
df[df.symbol.str.contains('ETH')]

In [ ]:
df[df.symbol.str.contains("BTC") & df.symbol.str.contains("USD")]

In [ ]:
df[df.symbol.str.contains("BTC") & df.symbol.str.contains("EUR")]

In [ ]:
last24 = client.get_ticker(symbol = 'BTCUSDT')
last24

In [ ]:
last24["openTime"]

In [ ]:
pd.to_datetime(last24["openTime"], unit="ms")

In [ ]:
pd.to_datetime(last24["closeTime"], unit="ms")

In [ ]:
open_price = float(last24["openPrice"])
open_price

In [ ]:
high_price = float(last24["highPrice"])
high_price

In [ ]:
low_price = float(last24["lowPrice"])
low_price

In [ ]:
close_price = float(last24["lastPrice"])
close_price

In [ ]:
close_price - open_price

In [ ]:
(close_price / open_price - 1) * 100

## 과거 데이터 가져오기

In [ ]:
timestamp = client._get_earliest_valid_timestamp(symbol = "BTCUSDT", interval="1d")
timestamp

In [ ]:
pd.to_datetime(timestamp, unit="ms")

In [ ]:
bars = client.get_historical_klines(symbol="BTCUSDT", interval="1d", start_str=timestamp, limit=1000)
bars

In [ ]:
df = pd.DataFrame(bars)
df

In [ ]:
df["Date"] = pd.to_datetime(df.iloc[:,0], unit="ms")
df

In [ ]:
df.columns = ["Open time", "Open", "High", "Low", "Close",
              "Volume", "Close time", "Quote asset volume",
              "Number of trades", "Taker buy base asset volume",
              "Taker buy quote asset volume", "Ignore", "Date"]
df

In [ ]:
df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
df

In [ ]:
df.set_index("Date", inplace=True)
df

In [ ]:
df.info()

In [ ]:
for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors="coerce")

In [ ]:
df.info()

In [ ]:
df

In [ ]:
def get_history(symbol, interval, start, end = None):
    bars = client.get_historical_klines(symbol=symbol, interval = interval,
                                        start_str = start, end_str = end, limit = 100)
    df = pd.DataFrame(bars)
    df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
    df.columns = ["Open time", "Open", "High", "Low", "Close",
              "Volume", "Close time", "Quote asset volume",
              "Number of trades", "Taker buy base asset volume",
              "Taker buy quote asset volume", "Ignore", "Date"]
    df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
    df.set_index("Date", inplace = True)
    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors="coerce")
        
    return df

In [ ]:
timestamp

In [ ]:
# df = get_history(symbol = "BTCUSDT", interval="1d", start = timestamp)
# df

In [ ]:
df.info()

In [ ]:
df = get_history(symbol="BTCUSDT", interval="1d", start="2021-01-01", end="2021-06-30")
df

In [ ]:
df = get_history(symbol="BTCUSDT", interval="1w", start="2021-01-01", end="2021-06-30")
df

In [ ]:
df = get_history(symbol="BTCUSDT", interval="1M", start="2021-01-01", end="2021-06-30")
df

In [ ]:
df = get_history(symbol="BTCUSDT", interval="1M", start="2021-01-01")
df

In [ ]:
df = get_history(symbol="BTCUSDT", interval="1h", start="2021-10-01", end="2021-10-05")
df

In [ ]:
df = get_history(symbol="BTCUSDT", interval="1h", start="2021-10-01 10:00:00", end="2021-10-05 16:00:00")
df

In [ ]:
# df = get_history(symbol="BTCUSDT", interval="1m", start="2021-10-01 10:29:00", end="2021-10-05 16:55:00")
# df

In [ ]:
from datetime import datetime, timedelta

In [ ]:
now = datetime.utcnow()
now

In [ ]:
two_hours_before = now - timedelta(hours = 2)
two_hours_before

In [ ]:
str(two_hours_before)

In [ ]:
df = get_history(symbol="BTCUSDT", interval="1m", start=str(two_hours_before))
df

In [ ]:
import pandas as pd

In [ ]:
url = "/Users/taemin/python-binance/BTCUSDT-1h-2021-09.csv"
url

In [ ]:
df = pd.read_csv(url, header = None)
df

In [ ]:
df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
df.columns = ["Open time", "Open", "High", "Low", "Close",
          "Volume", "Close time", "Quote asset volume",
          "Number of trades", "Taker buy base asset volume",
          "Taker buy quote asset volume", "Ignore", "Date"]
df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
df.set_index("Date", inplace = True)
for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors="coerce")
df

In [ ]:
df.info()

## 실시간 마켓 데이터

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from binance import ThreadedWebsocketManager
import pandas as pd

In [ ]:
def stream_data(msg):
    ''''''
    print(msg)

In [ ]:
twm = ThreadedWebsocketManager()
twm.start()

In [ ]:
twm

In [ ]:
twm.start_symbol_miniticker_socket(callback=stream_data, symbol="BTCUSDT")

In [ ]:
twm.stop()

In [ ]:
def stream_data(msg):
    time = pd.to_datetime(msg["E"], unit="ms")
    price = msg["c"]
    
    print("Time: {} | price: {}".format(time, price))

In [ ]:
twm = ThreadedWebsocketManager()
twm.start()

In [ ]:
twm.start_symbol_miniticker_socket(callback=stream_data, symbol="BTCUSDT")

In [ ]:
twm.stop()

In [ ]:
from binance import ThreadedWebsocketManager
import pandas as pd

In [ ]:
df = pd.DataFrame(columns=["Open", "High", "Low", "Close", "Volume", "Complete"])
df

In [ ]:
def stream_candles(msg):
    event_time = pd.to_datetime(msg["E"], unit="ms")
    start_time = pd.to_datetime(msg["k"]["t"], unit="ms")
    first = float(msg["k"]["o"])
    high = float(msg["k"]["h"])
    low = float(msg["k"]["l"])
    close = float(msg["k"]["c"])
    volume = float(msg["k"]["v"])
    complete = msg["k"]["x"]
    
    print("Time: {} | Price: {}".format(event_time, close))
    
    df.loc[start_time] = [first, high, low, close, volume, complete]

In [ ]:
twm = ThreadedWebsocketManager()
twm.start()

In [ ]:
twm.start_kline_socket(callback=stream_candles, symbol="BTCUSDT", interval="1m")

In [ ]:
df

In [ ]:
twm.stop()

In [ ]:
df.info()

## 모의 주문 생성

In [ ]:
from binance.client import Client

In [ ]:
api_key = "dLgGHojVxEF3ulipNtYNSh4wKacDnZjkzOhR5sbx431ditIjgF7iqbbLniAFmUQ6"
secret_key = pd.read_table("secret_key.txt").columns[0]

In [ ]:
client = Client(api_key=api_key, api_secret=secret_key, tld="com")

In [ ]:
order = client.create_test_order(symbol="BTCEUR", side = "BUY", type="MARKET", quantity=0.1)

In [ ]:
order